# Comparison of pandapower, pypower and Direct Distribution Load Flow (DDLF) results

In [24]:
import numpy as np

import pandapower as pp
import pandapower.networks as ppnets
import pandapower.converter as ppconv

## pandapower

In [11]:
# load network and run power flow
net = ppnets.panda_four_load_branch()
pp.runpp(net)
print (net.res_bus)

      vm_pu  va_degree        p_kw     q_kvar
0  1.000000   0.000000 -123.666645 -43.409193
1  0.987595  -0.971588    0.000000   0.000000
2  0.977858  -0.982721   30.000000  10.000000
3  0.970535  -0.991218   30.000000  10.000000
4  0.965642  -0.996953   30.000000  10.000000
5  0.963192  -0.999842   30.000000  10.000000


In [6]:
ppc = ppconv.to_ppc(net)

## pypower N-R

In [8]:
import pypower.api as pypow
results, success = pypow.runpf(ppc)
# default Newton-Raphson did not converge...

PYPOWER Version 5.0.0, 29-May-2015 -- AC Power Flow (Newton)


Newton's method power did not converge in 10 iterations.

Did not converge (0.10 seconds)

|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              6     Total Gen Capacity       0.0       -1000000000.0 to 1000000000.0
Generators         1     On-line Capacity         0.0       -1000000000.0 to 1000000000.0
Committed Gens     1     Generation (actual)      1.9               6.0
Loads              4     Load                     0.1               0.0
  Fixed            4       Fixed                  0.1               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           5     Losses (I^2 * Z)         4.27              6.

/Users/jko/anaconda/lib/python2.7/site-packages/pypower/ext2int.py:152: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  o["bus"]["e2i"] = zeros(max(o["bus"]["i2e"]) + 1)


### Pay attention in obvious difference between pandapower and pypower ppc case: in pandapower loads are connected to buses 2,3,4,5, while in pypower ppc to 1,2,3,4

## pypower Gauss-Seidel

In [9]:
# changing algorithm to Gauss-Seidel
ppopt = pypow.ppoption(PF_ALG=4)
results, success = pypow.runpf(ppc, ppopt=ppopt)

PYPOWER Version 5.0.0, 29-May-2015 -- AC Power Flow (Gauss-Seidel)


Gauss-Seidel power flow converged in 286 iterations.

Converged in 0.47 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              6     Total Gen Capacity       0.0       -1000000000.0 to 1000000000.0
Generators         1     On-line Capacity         0.0       -1000000000.0 to 1000000000.0
Committed Gens     1     Generation (actual)      0.1               0.0
Loads              4     Load                     0.1               0.0
  Fixed            4       Fixed                  0.1               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           5     Losses (I^2 * Z)         1.80              5.72
Tr

## Validate from ppc

In [20]:
# validation returns None obviously because default N-R Power Flow algorithm did not converge...
print ppconv.validate_from_ppc(ppc, net)

None


## Direct Distribution Load Flow (DDLF)

In [21]:
# direct load flow
from directDistPF import run_ddlf

V_DDLF = run_ddlf(ppc, epsilon=1.e-5)

 Direct LF successfully converged in 4

 A direct power flow executed in 0.00149393081665 s 
	 preparation time = 0.000639915466309 s


## Results comparison

In [46]:
res_df = net.res_bus[['vm_pu','va_degree']].rename(columns={'vm_pu':'vm_pandap','va_degree':'va_pandap'})

# adding pypower results
from pypower.idx_bus import VM, VA
res_df['vm_pypow'] = results['bus'][:, VM]
res_df['va_pypow'] = results['bus'][:, VA]

# adding directLF results
res_df['vm_DDLF'] = np.abs(V_DDLF)
res_df['va_DDLF'] = np.angle(V_DDLF,deg=True)
# adding 150 phase shift to DDLF result
res_df.loc[1:5,'va_DDLF'] -= 150

### Pypower and Direct LF have the results, while pandapower has different

In [56]:
print res_df[['vm_pandap','vm_pypow','vm_DDLF']], '\n'
print res_df[['va_pandap','va_pypow','va_DDLF']]

   vm_pandap  vm_pypow   vm_DDLF
0   1.000000  1.000000  1.000000
1   0.987595  0.963208  0.963208
2   0.977858  0.965657  0.965657
3   0.970534  0.970550  0.970550
4   0.965641  0.977874  0.977874
5   0.963192  0.987610  0.987610 

    va_pandap    va_pypow     va_DDLF
0    0.000000    0.000000    0.000000
1 -150.971565 -150.997211 -150.997210
2 -150.982699 -150.994322 -150.994321
3 -150.991195 -150.988587 -150.988586
4 -150.996930 -150.980091 -150.980090
5 -150.999819 -150.968958 -150.968957
